We consider the Peak-function given by the mixture of exponentially modified Gaussians as

$$P = \sum w_i T_i $$

with

In [2]:
import sympy as sp
x = sp.Symbol('x')
tau = sp.Symbol('tau_i')
sigma = sp.Symbol('sigma_i')
mu = sp.Symbol('mu_i')

T = sp.Symbol('T_i')
J_ = sp.Symbol('J_i')

J = sp.sqrt(2)*sp.exp(-(sigma/tau + (-mu+x)/sigma)**2/2)*sp.exp(sigma**2/(2*tau**2)+(-mu+x)/tau)/(2*sp.sqrt(sp.pi)*tau)

eq = sp.Eq(
    T,
    1/(2*tau) * sp.exp((x-mu)/tau + sigma**2/(2*tau**2)) * sp.erfc(((x-mu)/sigma + sigma/tau)/sp.sqrt(2))
)

eq

Eq(T_i, exp(sigma_i**2/(2*tau_i**2) + (-mu_i + x)/tau_i)*erfc(sqrt(2)*(sigma_i/tau_i + (-mu_i + x)/sigma_i)/2)/(2*tau_i))

In [3]:
eq_ddsigma = sp.Eq(
    sp.Derivative(T,sigma),
    sp.diff(eq.rhs,sigma).subs({eq.rhs:T,J:J_})
)

eq_ddtau = sp.Eq(
    sp.Derivative(T,tau),
    sp.diff(eq.rhs,tau).subs({eq.rhs:T,J:J_})
)

eq_ddmu = sp.Eq(
    sp.Derivative(T,mu),
    sp.diff(eq.rhs,mu).subs({eq.rhs:T,J:J_})
)


In [4]:
eq_ddsigma

Eq(Derivative(T_i, sigma_i), -J_i*(1/tau_i + (mu_i - x)/sigma_i**2) + T_i*sigma_i/tau_i**2)

In [5]:
eq_ddtau

Eq(Derivative(T_i, tau_i), J_i*sigma_i/tau_i**2 + T_i*(-sigma_i**2/tau_i**3 + (mu_i - x)/tau_i**2) - T_i/tau_i)

In [6]:
eq_ddmu

Eq(Derivative(T_i, mu_i), J_i/sigma_i - T_i/tau_i)

with

In [7]:
sp.Eq(J_,J.simplify())

Eq(J_i, sqrt(2)*exp((-mu_i**2 + 2*mu_i*x - x**2)/(2*sigma_i**2))/(2*sqrt(pi)*tau_i))

Hence, we can view the $P(x)$ as the dot product

$$P_{\theta}(x) = w_{\theta}^T T_{\theta} $$

where $T \in \mathbb{R}^{n \cdot 1}$ and $w \in \mathbb{R}^{n \cdot 1}$ and $\theta \in \mathbb{R}^{k \cdot 1}$ is a vector of parameters for the peak model.

Then, by product rule

$$\frac{\partial P_{\theta}}{\partial \theta} = w^T \frac{\partial T_{\theta}}{\partial \theta} + T\frac{\partial w_{\theta}}{\partial \theta}$$

which can help us to constrain the weights to sum to 1 (e.g. then include the gradient of that operation).

# Hessian

The Hessian of P, e.g. $\frac{\partial² P_{\theta}}{\partial \theta^T \partial \theta} $ involves computation using rank 3 tensors, however the end result is still a matrix, since $P \in \mathbb{R}$

$$ \frac{\partial² P_{\theta}}{\partial \theta^T \partial \theta} = $$

In [33]:
theta = sp.Array([sp.Symbol('theta_'+str(i)) for i in range(8)])
w = sp.Array([sp.Symbol('w_'+str(i)) for i in range(2)])
T = sp.Array([sp.Symbol('T_'+str(i)) for i in range(2)])

In [38]:
dwdtheta = sp.Matrix([
    [sp.Derivative(w[0],theta[i]) for i in range(8)],
    [sp.Derivative(w[1],theta[i]) for i in range(8)]
])

dTdtheta = sp.Matrix([
    [sp.Derivative(T[0],theta[i]) for i in range(8)],
    [sp.Derivative(T[1],theta[i]) for i in range(8)]
])

In [39]:
dwdtheta

Matrix([
[Derivative(w_0, theta_0), Derivative(w_0, theta_1), Derivative(w_0, theta_2), Derivative(w_0, theta_3), Derivative(w_0, theta_4), Derivative(w_0, theta_5), Derivative(w_0, theta_6), Derivative(w_0, theta_7)],
[Derivative(w_1, theta_0), Derivative(w_1, theta_1), Derivative(w_1, theta_2), Derivative(w_1, theta_3), Derivative(w_1, theta_4), Derivative(w_1, theta_5), Derivative(w_1, theta_6), Derivative(w_1, theta_7)]])

In [40]:
sp.Matrix(w).T*sp.Matrix(T)

Matrix([[T_0*w_0 + T_1*w_1]])

In [45]:
derivative = sp.Matrix(T).T*dwdtheta + sp.Matrix(w).T*dTdtheta

In [46]:
derivative[0]

T_0*Derivative(w_0, theta_0) + T_1*Derivative(w_1, theta_0) + w_0*Derivative(T_0, theta_0) + w_1*Derivative(T_1, theta_0)

# Positivity constraints

It is clear, that (for simplicity) all parameters of the model are strictily positive (in that the peaks have left-handed tailings).

Therefore, we apply the softplus transformation to each parameter

$$ f(x) = \ln{1 + e^x} $$

which is infinitely differentiable (and is available with numerically stable computation in a lot of languages).

Then, we can rewrite the derivatives and the hessian.

In [52]:
s = sp.Symbol('s')
t = sp.Symbol('t')
m = sp.Symbol('m')
sigma = sp.Symbol('sigma')
tau = sp.Symbol('tau')
mu = sp.Symbol('mu')

theta = sp.Array([s,t,m])

p = sp.Array([sp.log(1+sp.exp(s)),sp.log(1+sp.exp(t)),sp.log(1+sp.exp(m))])

T = 1/(2*p[1]) * sp.exp((x-p[2])/p[1] + p[0]**2/(2*p[1]**2)) * sp.erfc(((x-p[2])/p[0])+p[0]/p[1]/sp.sqrt(2))

T_ = sp.Symbol('T')

In [88]:
real_jacobian = sp.diff(T,theta)
real_hessian = sp.hessian(T,theta)

In [100]:
simple_jacobian = sp.Matrix(sp.diff(T.subs({p[0]:sigma,p[1]:tau,p[2]:mu}),sp.Array([sigma,tau,mu])))
simple_hessian = sp.Matrix(sp.diff(T.subs({p[0]:sigma,p[1]:tau,p[2]:mu}),sp.Array([sigma,tau,mu])))
jacobian_transform = sp.Matrix(sp.diff(p,theta))

In [96]:
jacobian_transform

Matrix([
[exp(s)/(exp(s) + 1),                   0,                   0],
[                  0, exp(t)/(exp(t) + 1),                   0],
[                  0,                   0, exp(m)/(exp(m) + 1)]])